# Hypothesis 26

### Freight value changes according to the product's weight and/or volume (height * length * width)

In [1]:
from pyspark.sql import SparkSession, functions as F, types as T
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.getOrCreate()

In [14]:
products_df = spark.read \
                .option('quote', '\"') \
                .option('escape', '\"') \
                .csv('./dataset/olist_products_dataset.csv', header=True, multiLine=True, inferSchema=True)

order_items_df = spark.read \
                      .option('quote', '\"') \
                      .option('escape', '\"') \
                      .csv('./dataset/olist_order_items_dataset.csv', header=True, multiLine=True, inferSchema=True)

print('Products Schema')
products_df.printSchema()

print('Order Items Schema')
order_items_df.printSchema()

df = order_items_df.join(products_df, 'product_id')

Products Schema
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)

Order Items Schema
root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [15]:
df.limit(10).toPandas()

,product_id,order_id,order_item_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,4244733e06e7ecb4970a6e2683c13e61,00010242fe8c5a6d1ba2dd792cb16214,1,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58,598,4,650,28,9,14
1,e5f2d52b802189ee658865ca93d83a8f,00018f77f2f0320c557190d7a144bdd3,1,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56,239,2,30000,50,30,40
2,c777355d18b72b67abbeef9df44fd0fd,000229ec398224ef6ca0657da4fc703e,1,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,59,695,2,3050,33,13,33
3,7634da152a4610f1595efa32f14722fc,00024acbcdf0a6daa1e931b038114c75,1,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,42,480,1,200,16,10,15
4,ac6c3623068f30de03045865e4e10089,00042b26cf59d7ce69dfabb4e55b4fd9,1,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,59,409,1,3750,35,40,30
5,ef92defde845ab8450f9d70c526ef70f,00048cc3ae777c65dbb7d2a0634bc1ea,1,6426d21aca402a131fc0a5d0960a3c90,2017-05-23 03:55:27,21.90,12.69,utilidades_domesticas,36,558,1,450,24,8,15
6,8d4f2bb7e93e6710a28f34fa83ee7d28,00054e8431b9d7675808bcb819fb4a32,1,7040e82f899a04d1b434b795a43b4617,2017-12-14 12:10:31,19.90,11.85,telefonia,52,815,1,200,27,5,20
7,557d850972a7d6f792fd18ae1400d9b6,000576fe39319847cbb9d288c5617fa6,1,5996cddab893a4652a15592fb58ab8db,2018-07-10 12:30:45,810.00,70.75,ferramentas_jardim,39,1310,3,13805,35,75,45
8,310ae3c140ff94b03219ad0adc3c778f,0005a1a1728c9d785b8e2b08b904576c,1,a416b6a846a11724393025641d4edd5e,2018-03-26 18:31:29,145.95,11.65,beleza_saude,59,493,1,2000,30,12,16
9,4535b0e1091c278dfd193e5a1d63b39f,0005f50442cb953dcd1d21e1fb923495,1,ba143b05f0110f0dc71ad71b4466ce92,2018-07-06 14:10:56,53.99,11.40,livros_tecnicos,52,1192,1,850,29,3,21


In [16]:
volume = F.udf(lambda x, y, z: float(x) * float(y) * float(z), T.DoubleType())

In [46]:
aux_df = df.select(F.col('product_id').alias('id'),
                   F.col('freight_value').alias('freight'),
                   F.col('product_weight_g').alias('weight'),
                   F.col('product_length_cm').alias('length'),
                   F.col('product_width_cm').alias('width'),
                   F.col('product_height_cm').alias('height')) \
            .filter(F.col('height').isNotNull())

aux_df = aux_df.withColumn('volume', volume('length', 'width', 'height'))

In [47]:
aux_df.show()

+--------------------+-------+------+------+-----+------+--------+
|                  id|freight|weight|length|width|height|  volume|
+--------------------+-------+------+------+-----+------+--------+
|4244733e06e7ecb49...|  13.29|   650|    28|   14|     9|  3528.0|
|e5f2d52b802189ee6...|  19.93| 30000|    50|   40|    30| 60000.0|
|c777355d18b72b67a...|  17.87|  3050|    33|   33|    13| 14157.0|
|7634da152a4610f15...|  12.79|   200|    16|   15|    10|  2400.0|
|ac6c3623068f30de0...|  18.14|  3750|    35|   30|    40| 42000.0|
|ef92defde845ab845...|  12.69|   450|    24|   15|     8|  2880.0|
|8d4f2bb7e93e6710a...|  11.85|   200|    27|   20|     5|  2700.0|
|557d850972a7d6f79...|  70.75| 13805|    35|   45|    75|118125.0|
|310ae3c140ff94b03...|  11.65|  2000|    30|   16|    12|  5760.0|
|4535b0e1091c278df...|   11.4|   850|    29|   21|     3|  1827.0|
|d63c1011f49d98b97...|   8.88|   950|    35|   20|    25| 17500.0|
|f177554ea93259a5b...|  12.98|   200|    16|   11|    16|  281

In [53]:
aux_df.stat.corr('freight', 'volume')

0.5872700711964617

In [49]:
aux_df.stat.corr('freight','weight')

0.6104202350972848